In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw

In [46]:
# Read data
crop_coordinate = pd.read_csv("data/Crop_Coordinates.csv")

# Hyper preameter 
H_radius, V_radius = 20, 42

# Horizontal margin, meaning how far each champion stay apart from each other on the same row
# S = self, O = oppoinent
H_margin_S, V_margin_S = 52, 34
H_margin_O, V_margin_O = 48.5, 25

# Grow of margin, from top to down
H_margin_Grow, V_margin_Grow = 0.01, 0

In [35]:
coordinate_gen_S = np.zeros((4, 7, 2))
coordinate_gen_O = np.zeros((4, 7, 2))

coordinate_gen_O[0,0,:] = (266, 70)
coordinate_gen_O[1,0,:] = (278, 70 + V_margin_O)
coordinate_gen_O[2,0,:] = (257, 70 + V_margin_O * (1 + V_margin_Grow)**1 * 2)
coordinate_gen_O[3,0,:] = (277, 70 + V_margin_O * (1 + V_margin_Grow)**2 * 3)

coordinate_gen_S[0,0,:] = (248, 176)
coordinate_gen_S[1,0,:] = (277, 176 + V_margin_S)
coordinate_gen_S[2,0,:] = (249, 176 + V_margin_S * (1 + V_margin_Grow)**1 * 2)
coordinate_gen_S[3,0,:] = (277, 176 + V_margin_S * (1 + V_margin_Grow)**2 * 3)

for i in range(4):
    H_margin_S_curr = H_margin_S * (1 + H_margin_Grow) ** i
    H_margin_O_curr = H_margin_O * (1 + H_margin_Grow) ** i
    coordinate_gen_S[i,:,0] = np.arange(coordinate_gen_S[i,0,0], 
                                        coordinate_gen_S[i,0,0] + H_margin_S_curr * 6 + 1, H_margin_S_curr)
    coordinate_gen_O[i,:,0] = np.arange(coordinate_gen_O[i,0,0], 
                                        coordinate_gen_O[i,0,0] + H_margin_O_curr * 6 + 1, H_margin_O_curr)
    coordinate_gen_S[i,:,1] = coordinate_gen_S[i,0,1]
    coordinate_gen_O[i,:,1] = coordinate_gen_O[i,0,1]

coordinate_gen_O = np.round(coordinate_gen_O, 0)
coordinate_gen_S = np.round(coordinate_gen_S, 0)

In [36]:
coordinate_gen_S_modeled = pd.DataFrame(data=coordinate_gen_S.reshape(4*7, -1),
             columns=["X", "Y"])
coordinate_gen_O_modeled = pd.DataFrame(data=coordinate_gen_O.reshape(4*7, -1),
             columns=["X", "Y"])

In [37]:
#coordinate_gen_O_modeled

### Testing Cropping Boundaries

In [40]:
# Display bounding box
img_path = "img/9/frame50790.jpg"

with Image.open(img_path) as im:
    
    draw = ImageDraw.Draw(im)
    
    for index, row in coordinate_gen_S_modeled.iterrows():
        X, Y = row[0], row[1] 
        #draw.rectangle(((X - H_radius, Y - V_radius), (X + H_radius, Y + V_radius)), outline="red")
        draw.ellipse(((X - 5, Y - 5), (X + 5, Y + 5)), fill = 'red', outline ='red')
        
    im.show()

In [48]:
# Cropped image into chunks
img_path = "img/9/frame55800.jpg"

with Image.open(img_path) as im:
    for index, row in coordinate_gen_O_modeled.iterrows():
        X, Y = row[0], row[1]
        cropper = (X - H_radius, Y - V_radius, X + H_radius, Y + V_radius)
        
        im.crop(cropper).save("img/9/frame55800/test_cropped%d.png" % index, "PNG")